# RAG Workshop #1 質問をTransform


## データを準備

### 使用データ
wikivoyageにある日本の観光地をURLでスクレイピングする

### ChromaDB というVector DBを使用してデータをEmbedding

In [1]:
from langchain_chroma import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
import chromadb

In [2]:
client = chromadb.PersistentClient(path="./my_chroma_db")

granular_collection = Chroma(
    client=client,
    collection_name="granular",
    embedding_function=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"),
)

granular_collection.reset_collection()

### HTMLSectionSplitterを使ってHTMLをスクレイプし、chromadbの中にINGESTする

In [3]:
from langchain_text_splitters import HTMLSectionSplitter
from langchain_community.document_loaders import AsyncHtmlLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
destinations = [
    "Tokyo",
    "Hiroshima",
    "Kanazawa",
    "Kyoto",
    "Nagasaki",
    "Nara",
    "Osaka",
    "Sapporo",
    "Sendai",
]

wikivoyage_root_url = "https://en.wikivoyage.org/wiki"


In [5]:
destination_urls = [f"{wikivoyage_root_url}/{d}" for d in destinations]

In [6]:
headers_to_split_on = [("h1", "Header 1"), ("h2", "Header 2")]
html_section_splitter = HTMLSectionSplitter(headers_to_split_on=headers_to_split_on)

In [7]:
def split_docs_into_granular_chunks(docs):
    all_chunks = []
    for doc in docs:
        html_string = doc.page_content
        temp_chunks = html_section_splitter.split_text(html_string)
        h2_temp_chunks = [
            chunk for chunk in temp_chunks if "Header 2" in chunk.metadata
        ]
        all_chunks.extend(h2_temp_chunks)

    return all_chunks


In [8]:
for destination_url in destination_urls:
    html_loader = AsyncHtmlLoader(destination_url)
    docs = html_loader.load()

    for doc in docs:
        print(doc.metadata)
        granular_chunks = split_docs_into_granular_chunks(docs)
        granular_collection.add_documents(documents=granular_chunks)


Fetching pages: 100%|##########| 1/1 [00:00<00:00,  1.86it/s]


{'source': 'https://en.wikivoyage.org/wiki/Tokyo', 'title': 'Tokyo – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|##########| 1/1 [00:00<00:00,  1.88it/s]


{'source': 'https://en.wikivoyage.org/wiki/Hiroshima', 'title': 'Hiroshima – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|##########| 1/1 [00:00<00:00,  2.08it/s]


...

{'source': 'https://en.wikivoyage.org/wiki/Osaka', 'title': 'Osaka – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|##########| 1/1 [00:00<00:00,  2.18it/s]


{'source': 'https://en.wikivoyage.org/wiki/Sapporo', 'title': 'Sapporo – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|##########| 1/1 [00:00<00:00,  2.19it/s]


{'source': 'https://en.wikivoyage.org/wiki/Sendai', 'title': 'Sendai – Travel guide at Wikivoyage', 'language': 'en'}


## Rewrite-retrieve-readを使おう

### 従来のRAGの問題点
- ユーザーの質問が不明瞭な場合、適切なドキュメントを検索できないことがある。
- それを解決するために生まれたのが Rewrite-Retrieve-Readという手法

### ユーザーからの質問でドキュメントを検索してみる

-  Scoreは高いほど検索条件により近い

In [9]:
user_question = "北海道で楽しめることは？"
improved_results = granular_collection.similarity_search_with_score(
    query=user_question, k=4
)
for i, (doc, score) in enumerate(improved_results):
    print("-" * 100)
    print(f"🐯 Result {i + 1}:")
    print(f"💫 Score: {score:.2f}")
    print(doc)

----------------------------------------------------------------------------------------------------
🐯 Result 1:
💫 Score: 1.48
page_content='Contents 
 
 
 
 
 
 
 
 1   Understand 
 
 
 
 
 1.1   History 
 
 
 
 
 
 
 1.2   Climate 
 
 
 
 
 
 
 1.3   Tourist information site 
 
 
 
 
 
 
 
 
 2   Get in 
 
 
 
 
 2.1   By plane 
 
 
 
 
 
 
 2.2   By train 
 
 
 
 
 
 
 2.3   By bus 
 
 
 
 
 
 
 2.4   By boat 
 
 
 
 
 
 
 
 
 3   Get around 
 
 
 
 
 3.1   By subway 
 
 
 
 
 
 
 3.2   By bus 
 
 
 
 
 
 
 3.3   By bicycle 
 
 
 
 
 
 
 3.4   On foot 
 
 
 
 
 
 
 
 
 4   See 
 
 
 
 
 
 
 5   Do 
 
 
 
 
 
 
 6   Buy 
 
 
 
 
 
 
 7   Eat 
 
 
 
 
 7.1   Japanese 
 
 
 
 
 
 
 7.2   International 
 
 
 
 
 
 
 7.3   Vegetarian 
 
 
 
 
 
 
 7.4   Halal 
 
 
 
 
 
 
 
 
 8   Drink 
 
 
 
 
 8.1   Coffee 
 
 
 
 
 
 
 8.2   Alcohol 
 
 
 
 
 
 
 
 
 9   Learn 
 
 
 
 
 9.1   Buddhist meditation 
 
 
 
 
 
 
 9.2   Japanese language 
 
 
 
 
 
 
 
 
 10   Sleep 
 
 
 
 
 10.1   Budge

結果： 質問が悪いと引っ張ってこられるChunkもひどい

### ユーザーの質問を書き換えたらどうなるか…

### 質問を書き換えてくれる Rewriter chain を setup

In [10]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [11]:
llm = ChatOllama(model="gemma3:4b")

‼️ ユーザーからのインプットは日本語、最終アウトプットは日本、内部は英語が望ましい。

In [12]:
rewriter_prompt_template = """
Generate search query for the Chroma DB vector store from a user question, allowing for a more accurate response through semantic search.
Just return the revised Chroma DB query, with quotes around it. 

User question: {user_question}
Revised Chroma DB query:
"""

rewriter_prompt = ChatPromptTemplate.from_template(rewriter_prompt_template)

In [13]:
rewriter_chain = rewriter_prompt | llm | StrOutputParser()


### 書き直された質問で情報をchromadbから回収する

In [ ]:
search_query = rewriter_chain.invoke({"user_question": user_question})
print(search_query)

"北海道 おすすめ 観光 物産 活动"



In [15]:
improved_results = granular_collection.similarity_search_with_score(
    query=user_question, k=4
)
for i, (doc, score) in enumerate(improved_results):
    print("-" * 100)
    print(f"🐯 Result {i + 1}:")
    print(f"💫 Score: {score:.2f}")
    print(doc)

----------------------------------------------------------------------------------------------------
🐯 Result 1:
💫 Score: 1.48
page_content='Contents 
 
 
 
 
 
 
 
 1   Understand 
 
 
 
 
 1.1   History 
 
 
 
 
 
 
 1.2   Climate 
 
 
 
 
 
 
 1.3   Tourist information site 
 
 
 
 
 
 
 
 
 2   Get in 
 
 
 
 
 2.1   By plane 
 
 
 
 
 
 
 2.2   By train 
 
 
 
 
 
 
 2.3   By bus 
 
 
 
 
 
 
 2.4   By boat 
 
 
 
 
 
 
 
 
 3   Get around 
 
 
 
 
 3.1   By subway 
 
 
 
 
 
 
 3.2   By bus 
 
 
 
 
 
 
 3.3   By bicycle 
 
 
 
 
 
 
 3.4   On foot 
 
 
 
 
 
 
 
 
 4   See 
 
 
 
 
 
 
 5   Do 
 
 
 
 
 
 
 6   Buy 
 
 
 
 
 
 
 7   Eat 
 
 
 
 
 7.1   Japanese 
 
 
 
 
 
 
 7.2   International 
 
 
 
 
 
 
 7.3   Vegetarian 
 
 
 
 
 
 
 7.4   Halal 
 
 
 
 
 
 
 
 
 8   Drink 
 
 
 
 
 8.1   Coffee 
 
 
 
 
 
 
 8.2   Alcohol 
 
 
 
 
 
 
 
 
 9   Learn 
 
 
 
 
 9.1   Buddhist meditation 
 
 
 
 
 
 
 9.2   Japanese language 
 
 
 
 
 
 
 
 
 10   Sleep 
 
 
 
 
 10.1   Budge

###  RAG Chain を作成

In [16]:
from langchain_core.runnables import RunnablePassthrough

In [17]:
retriever = granular_collection.as_retriever()

rag_prompt_template = """
Given a question and some context, answer the question.
If you do not know the answer, just say I do not know.
Answer in bullet points in Japanese.


Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(rag_prompt_template)

rewrite_retrieve_read_rag_chain = (
    {
        "context": {"user_question": RunnablePassthrough()}
        | rewriter_chain
        | retriever,
        "question": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)


In [18]:
answer = rewrite_retrieve_read_rag_chain.invoke(user_question)
print(answer)

北海道で楽しめることはたくさんあります！以下にいくつか例を挙げます。

*   **観光スポット:**
    *   白い恋人パーク
    *   ニigata ラベンダー
    *   羊蹄山
    *   小樽運河
    *   富良野ラベンダー
*   **アクティビティ:**
    *   スキー・スノーボード
    *   サイクリング
    *   ハイキング
    *   乗馬
    *   カヌー
    *   釣り
*   **グルメ:**
    *   海鮮料理（特に新鮮なホタテやカニ）
    *   ジンギスカン
    *   ラーメン
    *   スープカレー
    *   乳製品
*   **イベント:**
    *   札幌国際ホカーンカーニバル
    *   札幌国際雪まつり
    *   富良野雪まつり

これらの他にも、北海道には魅力的な場所やアクティビティがたくさんあります。


### 結果
- まだまだ改良の余地がある。
- 質問がひどい場合の対応をしているだけ。

## Multiple query generation を使ってより正確にVector DB からDocumentをRetrieveしよう


### 従来のRAGの問題点
- ユーザーの質問にあるキーワードにだけ近い近似値スコアでDocumentをRetrieveしてしまう。
- それを解消するための手法が Multiple query generation

In [53]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.prompts import ChatPromptTemplate

from typing import List
from langchain_core.output_parsers import BaseOutputParser
from pydantic import BaseModel, Field

### MultiQueryRetriver の実装

### Promptの設定

In [20]:
multi_query_gen_prompt_template = """
You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines.
Original question: {question}
"""

multi_query_gen_prompt = ChatPromptTemplate.from_template(
    multi_query_gen_prompt_template
)

### multi-query parser の設定

In [21]:
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Parse out a question from each output line."""

    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines))


questions_parser = LineListOutputParser()

### 質問(Query)を複数作る Chain の作成

In [22]:
multi_query_gen_chain = multi_query_gen_prompt | llm | questions_parser

### どのようなQueryが生成されるかを確認

In [ ]:
multiple_queries = multi_query_gen_chain.invoke(user_question)
multiple_queries

### MultiQueryRetrieverを設定

In [25]:
basic_retriever = granular_collection.as_retriever()

multi_query_retriever = MultiQueryRetriever(
    retriever=basic_retriever,
    llm_chain=multi_query_gen_chain,
    parser_key="lines",
)

### multi_query retrieverの出力を確認

In [ ]:
retrieved_docs = multi_query_retriever.invoke(user_question)
retrieved_docs

# Step-back question の実装

### 従来のRAGの問題点
- 詳細な質問をしたときに、特定のものを検索してしまうことによって、RetrieveできるDocumentが減ってしまう。
- そのために生まれた手法が Step-back Question

### Step-back question を生成するための準備

In [ ]:
step_back_prompt_template = """
Generate a less specific question (aka Step-back question) for the following detailed question, so that a wider context can be retrieved.
Detailed question: {detailed_question}
Step-back question:
"""

step_back_prompt = ChatPromptTemplate.from_template(step_back_prompt_template)

In [54]:
step_back_question_gen_chain = step_back_prompt | llm | StrOutputParser()

### step-back-question generation chain によって得られる質問を確認

In [ ]:
step_back_question = step_back_question_gen_chain.invoke(user_question)

In [34]:
step_back_question

'Here are a few options for a step-back question, aiming for a broader context than "北海道で楽しめることは？" (What can you enjoy in Hokkaido?)\n\n**Option 1 (Focus on travel/experience):**\n\n"What are some popular travel destinations or activities for tourists visiting Japan?" \n\n**Option 2 (More open-ended):**\n\n"What are some interesting things to do or see in Japan?"\n\n**Option 3 (Slightly more specific, but still broader):**\n\n"What are some of the most well-known attractions and experiences in the northern regions of Japan?"\n\n**Why these are step-back questions:**\n\n*   They move away from the specific location ("北海道") and focus on broader travel/tourism concepts.\n*   They are likely to trigger a wider range of responses and information, including things like Japanese culture, food, scenery, and general travel advice.\n\nI think **Option 1** is the strongest as it directly relates to the kind of information the original question is seeking.\n\nWould you like me to generate some oth

### Step-back question をRAG chain に導入する

In [ ]:
retriever = granular_collection.as_retriever()

rag_prompt_template = """
Given a question and some context, answer the question.
If you do not know the answer, just say I do not know.
If the information you get is in English, translate it to Japanese.
You must answer in bullet points in Japanese.
Give me as much detail as possible.

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(rag_prompt_template)

step_back_question_rag_chain = (
    {
        "context": {"detailed_question": RunnablePassthrough()}
        | step_back_question_gen_chain
        | retriever,
        "question": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)
user_question

'北海道で楽しめることは？'

In [ ]:
answer = step_back_question_rag_chain.invoke(user_question)
print(answer)

北海道で楽しめることはたくさんあります！ 

主な観光スポットやアクティビティとしては、以下のようなものが挙げられます。

*   **観光スポット:**
    *   札幌：時計台、大通公園、北海道庁旧本館など
    *   小樽：運河、ガラス工芸など
    *   富良野：ラベンダー畑（夏）、スキー場（冬）
    *   美瑛：パッチワークの丘
    *   洞爺湖：観光スポット、温泉
    *   旭山動物園
*   **アクティビティ:**
    *   スキー、スノーボード（冬）
    *   ラフティング、カヌー（夏）
    *   乗馬
    *   サイクリング
    *   グルメ（ラーメン、ジンギスカン、海鮮など）
    *   温泉巡り

また、上記以外にも、各地に様々な観光スポットやアクティビティがあります。



##  Rewrite-Retrieve-Readの利点
- ざっくりの質問からよりVector DBが検索しやすい、正確な近しいDocumentをRetrieveしてくれるような質問に書き換えてくれる

## Multi Query Generationの利点
- ユーザーに質問から複数の質問を色々な角度から生成してくることにより、近似値計算の弊害を弱めてより近しいDocumentをRetrieveするようにしてくれる
## Step back Questionの利点
- 詳しすぎる質問をしたときに、多くのDocumentをRetriveしてくれるようにスコープを広げた質問に変換してくれる

## 問題点は？
- 同じ質問をしても、答えの質がばらばら（non-deterministic）
  - 解決策： LoRAなどのチューニングが必要 
- 質問によって手法を変える必要がある
  - 解決策： LangGraphなどで条件分岐を行う必要がある 
- そもそも、間違った土地の情報が入ってしまう
  - 解決策： Chunkのなかにメタキーワードを埋め込み、フィルターする必要がある 